<p><span style="background-color: #ccffcc;"><strong>This notebook gives a SAGE function to compute the partial dual of an $l$-modular lattice given by its Gram matrix.</strong></span></p>
<p><span style="background-color: #ccffcc;"><strong>It then looks for strongly modular lattices among the 14- and 15-modular lattices of dimension 4.</strong></span></p>

<p>The partial dual of a lattice of level $l$, by definition [<span id="cell_outer_0">Atkin-Lehner eigenforms and strongly modular lattices", by H.-G. Quebbemann</span>], in terms of generator matrix is<br />$D_d\Lambda = \sqrt{d}\left( \frac{1}{d}\Lambda \cap \Lambda^*\right) $ <br />for $d$ a divisor of $l$.</p>
<p>To compute a partial dual, we need a basis for the intersection of two bases $B_1$ and $B_2$. For that, we compute dual bases $B_1^*$ and $B_2^*$ respectively, then we compute the span of  $B_1^*$ and $B_2^*$ and compute a Hermite normal form to obtain a basis, after which we compute the dual of this basis (Omeara, An introduction to Quadratic Forms, 82F).</p>
<p>From a Gram matrix $G$, we obtain a generator matrix using a Cholesky decomposition, that is we write</p>
<p>$G=LL^T$ where $L$ is a lower triangular matrix.</p>
<p>We note that</p>
<p>$$l_{11}=\sqrt{g_{11}}, l_{j1}=g_{1j}/l_{11},~j\geq2$$</p>
<p>thus the first colum will have integer coefficients if we factor out $1/l_{11}$.</p>
<p>Then</p>
<p>$$l_{22} =\sqrt{a_{22}-a_{12}^2/l_{11}^2},~l_{j2}=(a_{2j}-l_{j1}l_{21})/l_{22},~j\geq 3$$</p>
<p>and again, we notice that since $l_{j1}l_{21}$ has $l_{11}^2$ which is an integer at the denominator, if we factor out $1/l_{22}$, we get rational coefficients. If we also factor out $1/l_{11}^2$, we get integer coefficients.</p>
<p>Then $l_{33}$ is again a square root, and</p>
<p>$$l_{43}=(1/l_{33})(a_{34}-l_{41}l_{31}-l_{42}l_{32}).$$</p>
<p>The second term has $l_{11}^2$ as the denominator. The third term has $l_{22}^2$ at the denominator.</p>
<p>A Hermite normal form is computed on an integer matrix. The above discussion tells us that the basis matrix can be multiplied by a diagonal matrix, such that the product is an integer matrix.</p>

In [4]:
def partial_dual(G,d):
#take as input an nxn Gram matrix of an l-modular lattice, and d divides l 
#output: a generator matrix (rows are basis vectors) and a Gram matrix for the partial dual D_d
    n = G.nrows()
    #B is a generator matrix
    B = G.cholesky()
    D = matrix(RR,n)
    D[0,0] = B[0,0]
    for i in range(1,n):
        D[i,i] = B[i,i]*D[i-1,i-1]^2
    C = block_matrix([[(B/d).inverse().transpose()],[B]])
    #C*D is an integer matrix, compute the Hermite Normal Form on it
    CD = matrix(ZZ,2*n,n)
    CDtmp = C*D
    for i in range(0,2*n):
        for j in range(0,n):
            CD[i,j] = round(CDtmp[i,j])
    #U[1] contains the unimodular matrix that computes the reduced basis        
    U = (CD).hermite_form(transformation=true)
    #reduced basis for the span of C
    UC = (U[1]*C)[0:n,:]
    #basis for the partial dual
    B_partdual = sqrt(d*(1.0))*UC.inverse().transpose()
    #Gram matrix for the partial dual
    G_partdualtmp = B_partdual*B_partdual.transpose()
    G_partdual = matrix(ZZ,n)
    for i in range(0,n):
        for j in range(0,n):
            G_partdual[i,j] = round(G_partdualtmp[i,j])
    #use sqrt(d) for a symbolic output, and sqrt(d*(1.0)) for a numeric one
    return B_partdual, G_partdual

<p>We check with the lattice $A_2$ which is 3-modular.</p>
<p>We use $d=1,3$ and each time check that the bases are equivalent.</p>
<p>In the first method, the unimodular transformation is guessed.</p>
<p>In the second method, it is given as an output from the LLL_gram function.</p>

In [1]:
GA2 = matrix(ZZ, [[2,1],[1,2]])
CA2 = partial_dual(GA2,1)
U = matrix(ZZ,[[1,0],[0,-1]]) 
U*CA2[1]*U.transpose()

[2 1]
[1 2]

In [16]:
GA2 = matrix(ZZ, [[2,1],[1,2]])
CA2 = partial_dual(GA2,3)
U = CA2[1].LLL_gram()
U.transpose()*CA2[1]*U

[2 1]
[1 2]

<p><span style="font-size: xx-large; background-color: #ccffcc;">$l=15$</span></p>
<p>We consider the five 15-modular lattices in dimension 4 according to G.L. Nipp, and check with the first 15-modular lattice.</p>
<p>For $d=3$, the lattice is not equivalent to its partial dual.</p>
<p>This is because the partial dual is equivalent to the second 15-modular lattice.</p>

In [9]:
G1 = matrix(ZZ, [[2,0,1,0],[0,2,0,1],[1,0,8,0],[0,1,0,8]])
C1 = partial_dual(G1,3)
#just a trick to print less digits in the matrices
MatPrint = MatrixSpace(RealField(prec=18),4,4)
U = C1[1].LLL_gram()
MatPrint(C1[0]), matrix(ZZ,[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]])*U.transpose()*C1[1]*U*matrix(ZZ,[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]])

(
[ 2.4495 0.00000 0.00000 0.00000]  [4 1 0 0]
[0.00000  2.4495 0.00000 0.00000]  [1 4 0 0]
[-1.2247 0.00000  1.5811 0.00000]  [0 0 4 1]
[0.00000 -1.2247 0.00000  1.5811], [0 0 1 4]
)

<p>We may also check that the determinant is right.</p>

In [18]:
det(C1[0])^2

225.000000000000

<p>We check the second 15-modular lattice. For $d=3$, the lattice and its partial dual are not equivalent.</p>
<p>We see that because the partial dual is equivalent to the first 15-modular lattice.</p>

In [20]:
G2 = matrix(ZZ, [[4,1,0,0],[1,4,0,0],[0,0,4,1],[0,0,1,4]])
C2 = partial_dual(G2,3)
U = C2[1].LLL_gram()
matrix(ZZ,[[1,0,0,0],[0,1,0,0],[0,0,-1,0],[0,0,0,1]])*U.transpose()*C2[1]*U*matrix(ZZ,[[1,0,0,0],[0,1,0,0],[0,0,-1,0],[0,0,0,1]])

[2 0 1 0]
[0 2 0 1]
[1 0 8 0]
[0 1 0 8]

<p>We check the 3rd one. For $d=3$, the lattice and its partial dual are equivalent.</p>

In [22]:
G3 = matrix(ZZ, [[2,1,0,0],[1,2,0,0],[0,0,10,5],[0,0,5,10]])
C3 = partial_dual(G3,3)
U = C3[1].LLL_gram()
matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]])*U.transpose()*C3[1]*U*matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]])

[ 2  1  0  0]
[ 1  2  0  0]
[ 0  0 10  5]
[ 0  0  5 10]

<p>For $d=5$, the lattice and its partial dual are equivalent. Therefore this lattice is strongly 15-modular.</p>

In [46]:
G3 = matrix(ZZ, [[2,1,0,0],[1,2,0,0],[0,0,10,5],[0,0,5,10]])
C3 = partial_dual(G3,5)
U = C3[1].LLL_gram()
matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]])*U.transpose()*C3[1]*U*matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]])

[ 2  1  0  0]
[ 1  2  0  0]
[ 0  0 10  5]
[ 0  0  5 10]

<p>We check the 4rth one for $d=3$. The lattice and its partial dual are equivalent.</p>

In [23]:
G4 = matrix(ZZ, [[2,0,0,1],[0,4,1,0],[0,1,4,0],[1,0,0,8]])
C4 = partial_dual(G4,3)
U = C4[1].LLL_gram()
matrix(ZZ,[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]])*U.transpose()*C4[1]*U*matrix(ZZ,[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]])

[2 0 0 1]
[0 4 1 0]
[0 1 4 0]
[1 0 0 8]

<p>For $d=5$, the lattice and its partial dual are equivalent. Therefore this is a strongly 15-modular lattice.</p>

In [48]:
G4 = matrix(ZZ, [[2,0,0,1],[0,4,1,0],[0,1,4,0],[1,0,0,8]])
C4 = partial_dual(G4,5)
U = C4[1].LLL_gram()
matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]])*U.transpose()*C4[1]*U*matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]])

[2 0 0 1]
[0 4 1 0]
[0 1 4 0]
[1 0 0 8]

<p>We check the 5th 15-modular lattice.</p>

In [25]:
G5 = matrix(ZZ, [[4,2,1,1],[2,4,-1,2],[1,-1,6,2],[1,2,2,6]])
C5 = partial_dual(G5,3)
U = C5[1].LLL_gram()
U1 = matrix(ZZ,[[0,-1,0,0],[1,0,0,0],[0,0,0,1],[0,0,-1,0]])
print U1*U.transpose()*C5[1]*U*U1.transpose(), det(U1)

[ 4  2  1  1]
[ 2  4 -1  2]
[ 1 -1  6  2]
[ 1  2  2  6] 1

In [33]:
G5 = matrix(ZZ, [[4,2,1,1],[2,4,-1,2],[1,-1,6,2],[1,2,2,6]])
C5 = partial_dual(G5,5)
U = C5[1].LLL_gram()
U1 = matrix(ZZ,[[1,1,0,0],[1,0,0,0],[0,0,0,-1],[1,0,-1,0]])
print U1*U.transpose()*C5[1]*U*U1.transpose(), det(U1)

[ 4  2  1  1]
[ 2  4 -1  2]
[ 1 -1  6  2]
[ 1  2  2  6] 1

<p><span style="font-size: xx-large; background-color: #ccffcc;">$l=14$</span></p>
<p>We now check for 14-modular lattices. According to G.L. Nipp, there are five of them.</p>
<p>For the first lattice, we check for $d=2,7$.</p>

In [36]:
G6 = matrix(ZZ,[[2,0,1,1],[0,2,1,1],[1,1,8,1],[1,1,1,8]])
det(G6)
C6 = partial_dual(G6,7)
U = C6[1].LLL_gram()
matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])*U.transpose()*C6[1]*U*matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])

[2 0 1 1]
[0 2 1 1]
[1 1 8 1]
[1 1 1 8]

In [42]:
G6 = matrix(ZZ,[[2,0,1,1],[0,2,1,1],[1,1,8,1],[1,1,1,8]])
det(G6)
C6 = partial_dual(G6,2)
U = C6[1].LLL_gram()
matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,1]])*U.transpose()*C6[1]*U*matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,1]])

[2 0 1 1]
[0 2 1 1]
[1 1 8 1]
[1 1 1 8]

<p>The second 14-modular lattice is strongly modular.</p>

In [37]:
G7 = matrix(ZZ,[[2,1,0,0],[1,4,0,0],[0,0,4,2],[0,0,2,8]])
C7 = partial_dual(G7,2)
U = C7[1].LLL_gram()
matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]])*U.transpose()*C7[1]*U*matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[0,0,1,0],[0,0,0,-1]])

[2 1 0 0]
[1 4 0 0]
[0 0 4 2]
[0 0 2 8]

In [50]:
G7 = matrix(ZZ,[[2,1,0,0],[1,4,0,0],[0,0,4,2],[0,0,2,8]])
C7 = partial_dual(G7,7)
U = C7[1].LLL_gram()
U.transpose()*C7[1]*U

[2 1 0 0]
[1 4 0 0]
[0 0 4 2]
[0 0 2 8]

<p>For the 3rd lattice we check with $d=7$. Its partial dual is equivalent to the 5th 14-modular lattice.</p>
<p>Therefore this lattice is not strongly modular.</p>

In [38]:
G8 = matrix(ZZ,[[2,1,1,1],[1,2,0,1],[1,0,10,5],[1,1,5,10]])
C8 = partial_dual(G8,7)
U = C8[1].LLL_gram()
U1 = matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[1,0,1,0],[0,0,0,1]])
U1*U.transpose()*C8[1]*U*U1.transpose()

[4 2 2 2]
[2 4 0 2]
[2 0 6 3]
[2 2 3 6]

<p>For the 4rth lattice, we check for $d=2,7$.</p>

In [39]:
G9 = matrix(ZZ,[[4,1,1,0],[1,4,0,1],[1,0,4,-1],[0,1,-1,4]])
det(G9)
C9 = partial_dual(G9,2)
U = C9[1].LLL_gram()
matrix(ZZ,[[-1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])*U.transpose()*C9[1]*U*matrix(ZZ,[[-1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])

[ 4  1  0  1]
[ 1  4  1  0]
[ 0  1  4 -1]
[ 1  0 -1  4]

In [52]:
G9 = matrix(ZZ,[[4,1,1,0],[1,4,0,1],[1,0,4,-1],[0,1,-1,4]])
det(G9)
C9 = partial_dual(G9,7)
U = C9[1].LLL_gram()
matrix(ZZ,[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]])*U.transpose()*C9[1]*U*matrix(ZZ,[[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,-1]])

[ 4  1  0  1]
[ 1  4  1  0]
[ 0  1  4 -1]
[ 1  0 -1  4]

<p>The 5th 14-modular lattice has a dual for $d=7$ which is equivalent to the 3rd 14-modular lattice, therefore this lattice is not strongly modular.</p>

In [41]:
G10 = matrix(ZZ,[[4,2,2,2],[2,4,0,2],[2,0,6,3],[2,2,3,6]])
C10 = partial_dual(G10,7)
U = C10[1].LLL_gram()
U1 = matrix(ZZ,[[1,0,0,0],[0,-1,0,0],[1,0,1,0],[0,0,0,1]])
U1*U.transpose()*C10[1]*U*U1.transpose()

[ 2  1  1  1]
[ 1  2  0  1]
[ 1  0 10  5]
[ 1  1  5 10]